In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
iris = load_iris()
x_org, y_org = iris.data, iris.target

In [ ]:
# 입력 데이터에 관해서는 sepal length(0)와 petal length(2)만을 추출한다.
x_select = x_org[:, [0, 2]]
print("원래 데이터", x_select.shape, y_org.shape)

In [ ]:
# 학습 데이터 산점도 표시
x_t0 = x_select[y_org == 0]
x_t1 = x_select[y_org == 1]
x_t2 = x_select[y_org == 2]
plt.figure(figsize=(6,6))
plt.scatter(x_t0[:,0], x_t0[:,1], marker='x', c='k', s=50, label='0 (setosa)')
plt.scatter(x_t1[:,0], x_t1[:,1], marker='o', c='b', s=50, label='1 (versicolour)')
plt.scatter(x_t2[:,0], x_t2[:,1], marker='+', c='k', s=50, label='2 (virginica)')
plt.xlabel('sepal_length', fontsize=14)
plt.ylabel('petal_length', fontsize=14)
plt.xticks(size=14)
plt.yticks(size=14)
plt.legend(fontsize=14)
plt.show()

데이터 전처리

In [ ]:
# 더미 변수 추가
x_all = np.insert(x_select, 0, 1.0, axis=1)

In [ ]:
# y를 One-hot-Vector
from sklearn.preprocessing import OneHotEncoder
# one = OneHotEncoder(sparse=False, categories="auto")
ohe = OneHotEncoder(sparse=False)
y_work = np.c_[y_org]
y_all_one = ohe.fit_transform(y_work)
print("오리지널", y_org.shape)
print("2차 원화", y_work.shape)
print("원핫 인코딩", y_all_one.shape)

In [ ]:
# 학습 데이터와 검증 데이터를 분할
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test, y_train_one, y_test_one = train_test_split(
    x_all, y_org, y_all_one, train_size=75, test_size=75, random_state=123
)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, y_train_one.shape, y_test_one.shape)

In [ ]:
print("입력 데이터(x)")
print(x_train[:5, :])

In [ ]:
print("정답 데이터(y)")
print(y_train[:5])

In [ ]:
print("정답 데이터 (원핫 인코딩)")
print(y_train_one[:5, :])

학습용 변수 설정

In [ ]:
# 학습 대상의 선택
x, yt = x_train, y_train_one

예측 함수

In [ ]:
# 소프트맥스 함수 (9.7.3)
def softmax(x):
    x= x.T
    x_max = x.max(axis=0)
    # 입력값이 너무 크면 오버플로우가 발생할 수 있다.
    # exp(지수함수) 호출 전 오버플로우 발생을 방지하기 위해 입력 데이터의 최대값을 뺀다.
    x = x - x_max
    w = np.exp(x)
    return (w / w.sum(axis=0)).T

In [ ]:
# 예측값 게산 (9.7.1), (9.7.2)
def pred(x, W):
    return softmax(x @ W)

평가

In [ ]:
# 교차 엔트로피 함수 (9.5.1)
def cross_entropy(yt, yp):
    return -np.mean(np.sum(yt * np.log(yp), axis=1))

In [ ]:
# 모델을 평가하는 함수
from sklearn.metrics import accuracy_score

def evaluate(x_test, y_test, y_test_one, W):

    # 예측값 계산(확률값)
    yp_test_one = pred(x_test, W)

    # 확률값에서 예측 클래스(0, 1, 2)를 도출
    yp_test = np.argmax(yp_test_one, axis=1)

    # 손실함수 값 계산
    loss = cross_entropy(y_test_one, yp_test_one)

    # 정확도 산출
    score = accuracy_score(y_test, yp_test)
    return loss, score

초기화 처리

In [ ]:
# 표본 수
M = x.shape[0]

# 입력 차원 수(더미 변수 포함)
D = x.shape[1]

# 분류 대상 클래스 수
N = yt.shape[1]

# 반복 횟수
iters = 10000

# 학습률
alpha = 0.01

# 가중치 행렬의 초기 설정(모두가 1)
W = np.ones((D, N))

# 평가 결과 기록
history = np.zeros((0, 3))

주요 처리

In [ ]:
for k in range(iters):

    # 예측값 계산 (9.7.1), (9.7.2)
    yp = pred(x, W)

    # 오차 계산 (9.7.4)
    yd = yp - yt

    # 가중치 업데이트 (9.7.5)
    W = W - alpha * (x.T @ yd) / M

    if (k % 10 == 0):
        loss, score = evaluate(x_test, y_test, y_test_one, W)
        history = np.vstack((history, np.array([k, loss, score])))
        print("epoch = %d loss = %f score = %f" % (k, loss, score))

정확도 확인

In [ ]:
# 손실함수의 값과 정확도 점검
print("[초기 상태] 손실함수: %f, 정확도 : %f" %(history[0, 1], history[0, 2]))
print("[최종 상태] 손실함수: %f, 정확도 : %f" %(history[-1, 1], history[-1, 2]))

In [ ]:
# 학습 곡선 표시 (손실 함수)
plt.plot(history[:,0], history[:,1])
plt.grid()
plt.ylim(0,1.2)
plt.xlabel('iter', fontsize=14)
plt.ylabel('loss', fontsize=14)
plt.title('iter vs loss', fontsize=14)
plt.show()

In [ ]:
# 학습 곡선 표시 (정밀도)
plt.plot(history[:,0], history[:,2])
plt.ylim(0,1)
plt.grid()
plt.xlabel('iter', fontsize=14)
plt.ylabel('accuracy', fontsize=14)
plt.title('iter vs accuracy', fontsize=14)
plt.show()

In [ ]:
# 3차원 표시
from mpl_toolkits.mplot3d import Axes3D
x1 = np.linspace(4, 8.5, 100)
x2 = np.linspace(0.5, 7.5, 100)
xx1, xx2 = np.meshgrid(x1, x2)
xxx = np.array([np.ones(xx1.ravel().shape), 
    xx1.ravel(), xx2.ravel()]).T
pp = pred(xxx, W)
c0 = pp[:,0].reshape(xx1.shape)
c1 = pp[:,1].reshape(xx1.shape)
c2 = pp[:,2].reshape(xx1.shape)
plt.figure(figsize=(8,8))
ax = plt.subplot(1, 1, 1, projection='3d')
ax.plot_surface(xx1, xx2, c0, color='lightblue', 
    edgecolor='black', rstride=10, cstride=10, alpha=0.7)
ax.plot_surface(xx1, xx2, c1, color='blue', 
    edgecolor='black', rstride=10, cstride=10, alpha=0.7)
ax.plot_surface(xx1, xx2, c2, color='lightgrey', 
    edgecolor='black', rstride=10, cstride=10, alpha=0.7)
ax.scatter(x_t0[:,0], x_t0[:,1], 1, s=50, alpha=1, marker='+', c='k')
ax.scatter(x_t1[:,0], x_t1[:,1], 1, s=30, alpha=1, marker='o', c='k')
ax.scatter(x_t2[:,0], x_t2[:,1], 1, s=50, alpha=1, marker='x', c='k')
ax.set_xlim(4,8.5)
ax.set_ylim(0.5,7.5)
ax.view_init(elev=40, azim=70)

입력 변수를 4차원으로 변경

In [ ]:
# 더미 변수 추가
x_all2 = np.insert(x_org, 0, 1.0, axis=1)

In [ ]:
# 학습 데이터와 검증 데이터로 분할
from sklearn.model_selection import train_test_split

x_train2, x_test2, y_train, y_test, \
y_train_one, y_test_one = train_test_split(
    x_all2, y_org, y_all_one, train_size = 75, test_size = 75, random_state=123
)
print(x_train2.shape, x_test2.shape, y_train.shape, y_test.shape, y_train_one.shape, y_test_one.shape)

In [ ]:
print("입력 데이터(x)")
print(x_train2[:5, :])

In [ ]:
# 학습 대상의 선택
x, yt, x_test = x_train2, y_train_one, x_test2

In [ ]:
# 초기화 처리

# 표본 수
M = x.shape[0]
# 입력 차원 수(더미 변수를 포함)
D = x.shape[1]
# 분류 대상 클래스 수
N = yt.shape[1]

# 반복 횟수
iters = 10000

# 학습률
alpha = 0.01

# 가중치 행렬의 초기 설정(모두가 1)
W = np.ones((D, N)) 

# 평가 결과 기록
history = np.zeros((0, 3))

In [ ]:
# 주요처리(4차원 버전)
for k in range(iters):

    # 예측값 계산 (9.7.1) (9.7.2)
    yp = pred(x, W)

    # 오차 계산 (9.7.4)
    yd = yp - yt

    # 가중치 업데이트 (9.7.5)
    W = W - alpha * (x.T @ yd) / M

    if(k % 10 == 0):
        loss, score = evaluate(x_test, y_test, y_test_one, W)
        history = np.vstack((history, np.array([k, loss, score])))
        print("epoch = %d loss = %f score = %f" %(k, loss, score))

In [ ]:
print(history.shape)

In [ ]:
# 손실함수의 값과 정확도 점검
print ( '[초기 상태] 손실함수: %f, 정확도: %f'
    % (history[0,1], history[0,2]))
print( '[최종 상태] 손실함수: %f, 정확도: %f'
    % (history[-1,1], history[-1,2]))

In [ ]:
# 학습 곡선 표시 (손실 함수)
plt.plot(history[:,0], history[:,1])
plt.ylim(0,1.2)
plt.grid()
plt.xlabel('iter', fontsize=14)
plt.ylabel('loss', fontsize=14)
plt.title('iter vs loss', fontsize=14)
plt.show()

In [ ]:
# 학습 곡선 표시 (정밀도)
plt.plot(history[:,0], history[:,2])
plt.ylim(0,1)
plt.grid()
plt.xlabel('iter', fontsize=14)
plt.ylabel('accuracy', fontsize=14)
plt.title('iter vs accuracy', fontsize=14)
plt.show()